<a href="https://colab.research.google.com/github/Medynal/Pollution/blob/main/pollution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import os

repo_url = "https://github.com/Medynal/Pollution.git"
folder_path = "/content/Pollution"

if not os.path.exists(folder_path):
  !git clone {repo_url} {folder_path}
print(f"Current working directory: {os.getcwd()}")


Current working directory: /content


In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, classification_report, confusion_matrix
import joblib
from lightgbm import LGBMRegressor

In [35]:
cpollution_df = pd.read_csv('/content/Pollution/IPNBS/cleaned_pollution_dataset.csv', parse_dates=['Date']).drop(['Month name', 'Date'], axis= 1)
cpollution_df.head()

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,year,month,day
0,Ahmedabad,73.24,141.54,0.92,18.22,17.15,26.64,0.92,27.64,133.36,0.00,0.02,0.00,209.0,Poor,2015,1,1
1,Ahmedabad,73.24,141.54,0.97,15.69,16.46,26.64,0.97,24.55,34.06,3.68,5.50,3.77,209.0,Poor,2015,1,2
2,Ahmedabad,73.24,141.54,17.40,19.30,29.70,26.64,17.40,29.07,30.70,6.80,16.40,2.25,209.0,Poor,2015,1,3
3,Ahmedabad,73.24,141.54,1.70,18.48,17.97,26.64,1.70,18.59,36.08,4.43,10.14,1.00,209.0,Poor,2015,1,4
4,Ahmedabad,73.24,141.54,22.10,21.42,37.76,26.64,22.10,39.33,39.31,7.01,18.89,2.78,209.0,Poor,2015,1,5


In [36]:
#Cyclical Transformation
# Month
cpollution_df['month_sin'] = np.sin(2 * np.pi * cpollution_df['month']/12)
cpollution_df['month_cos'] = np.cos(2 * np.pi * cpollution_df['month']/12)
# Day
cpollution_df['day_sin'] = np.sin(2 * np.pi * cpollution_df['day']/7)
cpollution_df['day_cos'] = np.cos(2 * np.pi * cpollution_df['day']/7)

#encoding AQI bucket
bucket_encoder = LabelEncoder()
cpollution_df['AQI_Bucket_encoded'] = bucket_encoder.fit_transform(cpollution_df['AQI_Bucket'])

In [37]:
cpollution_df.head()

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,...,AQI,AQI_Bucket,year,month,day,month_sin,month_cos,day_sin,day_cos,AQI_Bucket_encoded
0,Ahmedabad,73.24,141.54,0.92,18.22,17.15,26.64,0.92,27.64,133.36,...,209.0,Poor,2015,1,1,0.5,0.866025,0.781831,0.623490,2
1,Ahmedabad,73.24,141.54,0.97,15.69,16.46,26.64,0.97,24.55,34.06,...,209.0,Poor,2015,1,2,0.5,0.866025,0.974928,-0.222521,2
2,Ahmedabad,73.24,141.54,17.40,19.30,29.70,26.64,17.40,29.07,30.70,...,209.0,Poor,2015,1,3,0.5,0.866025,0.433884,-0.900969,2
3,Ahmedabad,73.24,141.54,1.70,18.48,17.97,26.64,1.70,18.59,36.08,...,209.0,Poor,2015,1,4,0.5,0.866025,-0.433884,-0.900969,2
4,Ahmedabad,73.24,141.54,22.10,21.42,37.76,26.64,22.10,39.33,39.31,...,209.0,Poor,2015,1,5,0.5,0.866025,-0.974928,-0.222521,2


In [38]:
pollutants = ['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','Benzene','Toluene','Xylene']
labels = ['City','year','month_sin','month_cos', 'day_sin', 'day_cos', 'AQI'] + pollutants

#AQI bucket model feature and target
X = cpollution_df[labels]
target_AQIB= cpollution_df['AQI_Bucket_encoded']

#AQI model feature and target
X_= cpollution_df[labels].drop('AQI', axis=1)
target_AQI= cpollution_df['AQI']

In [39]:
# Train-test split for AQI bucket
X_train, X_test, y_train, y_test = train_test_split( X, target_AQIB, test_size=0.2, random_state=42)

# Train-test split for AQI
Xb_train, Xb_test, yb_train, yb_test = train_test_split( X_, target_AQI, test_size=0.2, random_state=42)

In [40]:
preprocessor = ColumnTransformer(transformers=[('city', OneHotEncoder(handle_unknown='ignore'), ['City']),
        ('others', 'passthrough', ['year','month_sin','month_cos', 'day_sin', 'day_cos','AQI'] + pollutants)])

#classification pipeline to predict AQI_Bucket
AQIB_model = Pipeline([('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(n_estimators=200, random_state=42))])

AQIB_model.fit(X_train, y_train)
y_pred = AQIB_model.predict(X_test)
print('Classification accuracy:', accuracy_score(y_test, y_pred))
print('Classification report:\n', classification_report(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

Classification accuracy: 0.9964448958862366
Classification report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       296
           1       1.00      1.00      1.00      2062
           2       0.99      0.99      0.99       666
           3       1.00      1.00      1.00      2033
           4       0.99      0.97      0.98       302
           5       0.98      0.99      0.99       548

    accuracy                           1.00      5907
   macro avg       0.99      0.99      0.99      5907
weighted avg       1.00      1.00      1.00      5907

Confusion matrix:
 [[ 293    0    0    3    0    0]
 [   0 2062    0    0    0    0]
 [   0    2  662    0    0    2]
 [   0    0    0 2033    0    0]
 [   0    1    1    0  293    7]
 [   0    0    3    0    2  543]]


In [41]:
preprocessor1 = ColumnTransformer([
    ('city', OneHotEncoder(handle_unknown='ignore'), ['City']),
    ('others', 'passthrough', ['year','month_sin','month_cos','day_sin','day_cos'] + pollutants)])
AQI_pipeline = Pipeline([
    ('preprocessor', preprocessor1),
    ('lgbm', LGBMRegressor(random_state=42))])
param_grid = {
    'lgbm__num_leaves': [31, 50, 70],
    'lgbm__max_depth': [-1, 10, 20],
    'lgbm__learning_rate': [0.01, 0.05, 0.1],
    'lgbm__n_estimators': [200, 500, 800]}
AQI_grid = GridSearchCV(
    estimator = AQI_pipeline,
    param_grid = param_grid,
    scoring = 'neg_root_mean_squared_error',
    cv = 3,
    n_jobs = -1,
    verbose = 2)

AQI_grid.fit(Xb_train, yb_train)

print("Best Parameters:", AQI_grid.best_params_)
print("Best Score (neg RMSE):", AQI_grid.best_score_)
print("Best RMSE:", -AQI_grid.best_score_)






Fitting 3 folds for each of 81 candidates, totalling 243 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3188
[LightGBM] [Info] Number of data points in the train set: 23624, number of used features: 43
[LightGBM] [Info] Start training from score 163.650779
Best Parameters: {'lgbm__learning_rate': 0.05, 'lgbm__max_depth': -1, 'lgbm__n_estimators': 500, 'lgbm__num_leaves': 31}
Best Score (neg RMSE): -46.13258886569613
Best RMSE: 46.13258886569613


In [42]:
best_AQI_model = AQI_grid.best_estimator_

In [43]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = AQI_grid.predict(Xb_test)
rmse = np.sqrt(mean_squared_error(yb_test, y_pred))
rmse


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


np.float64(42.08810279488696)